# Project 3: Where's Waldo?

Due Mar. 4th

So far, we've mostly focused on using imagery to do stuff for which it is better suited than a human: calculating camera locations from imagery, finding an optimal projective transform to stitch images together, and (soon) we'll be doing "structure from motion" in which we create 3D models of the world from collections of 2D images.  These are tasks primarily based around measuring things and doing calculations.  On the other side of the coin is object recognition (identifying the semantic content of a scene), and the best contemporary computer vision algorithms do object recognition at roughly the level of a 2 year old human (with some exceptions).  For this (mini-)project, we're going to delve into a topic that sort of straddles the line between these two general realms of computer vision.

As a motivating example, did you ever play the game Where's Waldo.  There are books filled with images like the following:
<img src='waldo_1.jpg'>
The objective, of course, is to find Waldo, the man in the red striped shirt and beanie wearing glasses.  He looks like this:
<img src='waldo_template.jpg'>
These scenes are (obviously) intended to have a bunch of visual clutter to make this task reasonably challenging.

Your task will be to come up with an algorithm that locates the template image (Waldo's face) and the target image (the larger scene).  This is called *template matching*, and it's a primitive form of feature recognition.  

## Implementation
### Template Matching
Template matching works in a way that is very similar to filtering:  slide the template image over every location in the target image, computing some sort of metric at each position.  In practice, one commonly used choice for an error metric is the one that you've already used for matching keypoint descriptors: z-normalized sum square error.  Another choice is [normalized cross-correlation](https://en.wikipedia.org/wiki/Cross-correlation#Normalization).  Once these metrics have been computed, simply find the argmin (for SSE) or argmax (for NCC), and this will be the location of the best match.  

**Your task is to implement template matching.  Use 'waldo_template.jpg' as the template and 'waldo_1.jpg' as the target image.  Where's Waldo? **

### Not so fast!!!  What about scale!
Oh, no.  As it turns out, the template I've provided is not the same scale as the Waldo in the image.  To deal with this, you'll need to create an image pyramid for the template (See Szeliski 3.5, and [Mubarak Shah's lecture on this topic](https://www.youtube.com/watch?v=KO7jJt0WHag&feature=youtu.be)).  This essentially just means creating a sequence of downsampled images of the template, and trying each one in hopes that one of the resulting down-scaled templates matches the feature in the target image.  **Create a sequence of templates with which to perform feature matching, each one 1/2 the resolution of the previous (so 1/4 the total number of pixels).  To avoid aliasing, before downsampling perform a $\sigma=1$ Gaussian Blur of the image.  Once you've built your image pyramid, find the argmin/max in 3 dimensions (u,v,template scale)**.

## Generalization
**Waldo appears in every Where's Waldo image (obviously).  Try using the same technique on 'waldo_2.jpg'.  Does the algorithm work?**  I confess that I pulled the image of waldo for the template directly from 'waldo_1.jpg', so for the correct scale, there is something close to an exact match (i.e. SSE=0).  However, Waldo, while easily recognizable to the human eye after undergoing the small scale deformations associated with artistic license, is not so easily recognizable via template matching.  We will return to a similar problem when discussing object recognition, and hopefully this example will motivate the need to come up with representations of objects (like Waldo) that are more robust.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import matplotlib.patches as patches
import math as mt
import time


class templateMatch(object):

    def __init__(self,baseImage,template,downsample):
        self.baseImage = plt.imread(baseImage).mean(axis=2)
        self.template = plt.imread(template).mean(axis=2)
        self.downsample = downsample
        self.matches = []

    
    def gauss_kernal(self,size, var):
        kernel = np.zeros(shape=(size,size))
        for i in range(size):
            for j in range(size):
                kernel[i][j] = mt.exp( -((i - (size-1)/2)**2 ))
        kernel = kernel / kernel.sum()
        return kernel
                                     
    def convolve(self,g,h): # h is kernel, g is the image
        
        I_gray_copy = np.zeros(shape=(len(g[:,1]),len(g[1,:])))
        u,v = h.shape
        sp = int(u/2)

        start = sp
        for i in range(start,len(g[:,1])-1):
            for j in range(start, len(g[i,:])-1):
                f = g[i-sp:i+sp, j-sp:j+sp] #FIXME
                # need another array to put value into
                total = h*f
                I_gray_copy[i][j] = sum(sum(total))
        return I_gray_copy

    def downSample(self,image,n=None):
        if n is None:
            n = int(self.downsample)
        pyramid = []
        for i in range(n):
            image_conv = self.convolve(image,self.gauss_kernal(4,1))
            pyramid.append(image_conv)
            image = image[::2,::2]
        return pyramid

    def templMatch(self,BaseImage,matchImage):
        TA = time.time()
        U,V = BaseImage.shape
        pyramid = self.downSample(matchImage)
        #count = 0 #FIXME
        #print("U,V : ",U,V)
        for i in range(len(pyramid)):
            subMatchImage = pyramid[i]
            mU,mV = subMatchImage.shape
            argmin = np.inf
            bestU,bestV = None,None
            #print("Cross corr test start\n")
            for j in range(0,U-mU):
                for k in range(0,V-mV):
                
                   # compare = self.SSE(BaseImage[j:mU+j,k:mV+k],matchImage)
                    compare = ((BaseImage[j:mU+j,k:mV+k] - subMatchImage)**2).sum()/(mU*mV)
                
                    if(compare < argmin):
                       #count += 1
                       #print("Better match found,count: ",count)
                       argmin = compare
                       bestU = j
                       bestV = k
                #print("j loop: ",j)
            self.matches.append((i+1,bestU,bestV,argmin))
            #print("Cross corr test end\n")
        self.matches.sort(key = lambda x:x[-1])
        print("Best location estimate: ", self.matches[0][1],self.matches[0][2])
        print("All estimates: \n")

        for i in range(int(self.downsample)):
            print(self.matches[i][0] ," " ,self.matches[i][1], " ", self.matches[i][2]," ",self.matches[i][3])

        print("Elapsed Time: ",time.time()-TA, "\n")              
        return self.matches

    def plotBestMatch(self,matches,baseImage,template):
        lev = matches[0][0]
        tx,ty = template.shape
        tx,ty = tx/lev, ty/lev
        bx,by = baseImage.shape
        X,Y = lev*matches[0][1], lev*matches[0][2] #location of best match in base image
        #coords for square
        cX,cY = X+(tx/2),Y+(ty/2) #center of square
        #p1,p2,p3,p4 = [X,Y], [X + tx/2,Y], [X + tx/2,Y + ty/2], [X ,Y + ty/2]
        fig,ax = plt.subplots(1)
        ax.imshow(self.baseImage)
        rect = patches.Rectangle([self.matches[0][1],self.matches[0][2]],ty,tx,linewidth=5,edgecolor='g',facecolor='none')
        ax.add_patch(rect)
        plt.show()
        #plt.savefig("WheresWaldo.jpg")

    def runAndPlot(self):
        match = self.templMatch(self.baseImage,self.template)
        self.plotBestMatch(match,self.baseImage,self.template)

                       
                       
                       
                       
                       
                       
                       
                       
                       
                       
                       
                       
                       
                       

In [ ]:
WheresWaldo = templateMatch()